In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

from keras import layers, optimizers, regularizers
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.models import Sequential

from keras.utils import plot_model
#from kt_utils import *
import keras.backend as K


from sklearn import preprocessing, model_selection 
from matplotlib.pyplot import imshow


# Modelling Helpers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score

# Modelling Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier


import keras
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.optimizers import SGD

In [ ]:
df = pd.read_csv('breast-cancer-wisconsin-data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.drop(['Unnamed: 32', 'id'], axis=1, inplace= True)

In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
pd.isnull(df).sum()

In [ ]:
df['diagnosis'].value_counts()

In [ ]:
df['diagnosis'].unique()

In [ ]:
def categorical_to_numeric_diagnosis(x):
    if x=='M':
        return 1
    if x=='B':
        return 0

In [ ]:
df['diagnosis']= df['diagnosis'].apply(categorical_to_numeric_diagnosis)

In [ ]:
df["diagnosis"].value_counts()

In [ ]:
sns.countplot(df['diagnosis'])

In [ ]:
#correlation map to see correlation between variables
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(df.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
#splitting data to X ve y
X = df.drop(['diagnosis'], axis = 1)
y = df['diagnosis']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_train,X_test,y_train,y_test = model_selection.train_test_split(X,y,test_size=0.2)

print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
# X_train = X_train.values
# X_test= X_test.values

In [ ]:
def getModel(arr):
    model=Sequential()
    for i in range(len(arr)):
        if i!=0 and i!=len(arr)-1:
            if i==1:
                model.add(Dense(arr[i],input_dim=arr[0],kernel_initializer='normal', activation='relu'))
            else:
                model.add(Dense(arr[i],activation='relu'))
    model.add(Dense(arr[-1],kernel_initializer='normal',activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer='rmsprop',metrics=['accuracy'])
    return model

In [ ]:
firstModel=getModel([30,50,1])

In [ ]:
import keras
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

In [ ]:
firstModel.fit(np.array(X_train),np.array(y_train),epochs=40,callbacks=[plot_losses])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
scores=firstModel.evaluate(np.array(X_val),np.array(y_val))

In [ ]:
print("Loss:",scores[0])
print("Accuracy",scores[1]*100)

In [ ]:
secondModel=getModel([30,100,1])
secondModel.fit(np.array(X_train),np.array(y_train),epochs=40,callbacks=[plot_losses])

In [ ]:
scores2=secondModel.evaluate(np.array(X_val),np.array(y_val))

In [ ]:
print(scores2)

In [ ]:
thirdModel=getModel([30,50,70,40,1])

In [ ]:
thirdModel.fit(np.array(X_train),np.array(y_train),epochs=100,callbacks=[plot_losses])

In [ ]:
scores3=thirdModel.evaluate(np.array(X_val),np.array(y_val))

In [ ]:
print(scores3)

In [ ]:
predY=firstModel.predict(np.array(X_test))
predY=np.round(predY).astype(int).reshape(1,-1)[0]

In [ ]:
from sklearn.metrics import confusion_matrix
m=confusion_matrix(predY,y_test)
tn, fn, fp, tp=confusion_matrix(predY,y_test).ravel()
m=pd.crosstab(predY,y_test)
print("Confusion matrix")
print(m)

In [ ]:
sens=tp/(tp+fn)
spec=tn/(tn+fp)
print("Senstivity:",sens)
print("Specificity:",spec)

# Method 2

In [3]:
#imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.cross_validation import  train_test_split
from matplotlib import pyplot
from sklearn import metrics
from sklearn.cross_validation import KFold, cross_val_score
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree
from time import time
from sklearn.preprocessing import StandardScaler

In [8]:
df= pd.read_csv('breast-cancer-wisconsin-data.csv')

In [9]:
df.drop(['Unnamed: 32', 'id'], axis=1, inplace= True)

In [10]:
def categorical_to_numeric_diagnosis(x):
    if x=='M':
        return 1
    if x=='B':
        return 0

In [11]:
df['diagnosis']= df['diagnosis'].apply(categorical_to_numeric_diagnosis)

In [12]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [15]:
features = list(df.columns[1:31])

In [18]:
features

['radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'radius_se',
 'texture_se',
 'perimeter_se',
 'area_se',
 'smoothness_se',
 'compactness_se',
 'concavity_se',
 'concave points_se',
 'symmetry_se',
 'fractal_dimension_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst',
 'fractal_dimension_worst']

In [21]:
XGBmodel = XGBClassifier()
X_train, X_test, y_train, y_test = train_test_split(df[features], df['diagnosis'].values, test_size=0.30, random_state=42)

In [22]:
XGBmodel.fit(X_train,y_train)
predictions = XGBmodel.predict(X_test)     

print ("accuracy" , metrics.accuracy_score(y_test, predictions))

accuracy 0.9649122807017544


/home/sara/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [25]:
model = Sequential()
model.add(Dense(input_dim=30, output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=30, units=2)`
  


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [26]:
scaler = StandardScaler()
model.fit(scaler.fit_transform(X_train.values), y_train)

Epoch 1/10
398/398 [==============================] - 0s 545us/step - loss: 0.7955 - acc: 0.5427
Epoch 2/10
398/398 [==============================] - 0s 79us/step - loss: 0.4515 - acc: 0.8065
Epoch 3/10
398/398 [==============================] - 0s 130us/step - loss: 0.3263 - acc: 0.9020
Epoch 4/10
398/398 [==============================] - 0s 99us/step - loss: 0.2696 - acc: 0.9246
Epoch 5/10
398/398 [==============================] - 0s 114us/step - loss: 0.2375 - acc: 0.9372
Epoch 6/10
398/398 [==============================] - 0s 88us/step - loss: 0.2164 - acc: 0.9447
Epoch 7/10
398/398 [==============================] - 0s 104us/step - loss: 0.2011 - acc: 0.9472
Epoch 8/10
398/398 [==============================] - 0s 108us/step - loss: 0.1896 - acc: 0.9497
Epoch 9/10
398/398 [==============================] - 0s 91us/step - loss: 0.1802 - acc: 0.9472
Epoch 10/10
398/398 [==============================] - 0s 100us/step - loss: 0.1726 - acc: 0.9497


In [27]:
y_prediction = model.predict_classes(scaler.transform(X_test.values))
print ("\n\naccuracy" , np.sum(y_prediction == y_test) / float(len(y_test)))

171/171 [==============================] - 0s 114us/step


accuracy 0.9649122807017544


In [28]:
model = Sequential()
model.add(Dense(input_dim=30, output_dim=30))
model.add(Dense(input_dim=30, output_dim=30))
model.add(Dense(input_dim=30, output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=30, units=30)`
  
/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=30, units=30)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=30, units=2)`
  after removing the cwd from sys.path.


In [29]:
model.fit(scaler.fit_transform(X_train.values), y_train)

Epoch 1/10
398/398 [==============================] - 0s 599us/step - loss: 0.5203 - acc: 0.7764
Epoch 2/10
398/398 [==============================] - 0s 98us/step - loss: 0.2473 - acc: 0.8970
Epoch 3/10
398/398 [==============================] - 0s 148us/step - loss: 0.1858 - acc: 0.9271
Epoch 4/10
398/398 [==============================] - 0s 71us/step - loss: 0.1588 - acc: 0.9372
Epoch 5/10
398/398 [==============================] - 0s 109us/step - loss: 0.1432 - acc: 0.9422
Epoch 6/10
398/398 [==============================] - 0s 118us/step - loss: 0.1330 - acc: 0.9523
Epoch 7/10
398/398 [==============================] - 0s 195us/step - loss: 0.1245 - acc: 0.9548
Epoch 8/10
398/398 [==============================] - 0s 176us/step - loss: 0.1182 - acc: 0.9623
Epoch 9/10
398/398 [==============================] - 0s 172us/step - loss: 0.1136 - acc: 0.9623
Epoch 10/10
398/398 [==============================] - 0s 105us/step - loss: 0.1091 - acc: 0.9648


In [30]:
y_prediction = model.predict_classes(scaler.transform(X_test.values))
print ("\n\naccuracy" , np.sum(y_prediction == y_test) / float(len(y_test)))

171/171 [==============================] - 0s 195us/step


accuracy 0.9941520467836257
